# 🎨 RWC-AI-STUDIO
### Advanced AI Image Generation & Editing Studio for Google Colab

**Features:**
- Text-to-Image Generation
- Image-to-Image Editing & Enhancement
- Multiple AI Models (DreamShaper, RunwayML, Dreamlike)
- Image Regeneration with Specific Modifications
- Google Drive Integration
- Public Access via ngrok
- Web Interface

---

In [1]:
# Install required packages for RWC-AI-STUDIO
!pip install --quiet diffusers transformers accelerate scipy xformers torchvision torch safetensors huggingface_hub Flask pyngrok pillow flask-cors pydrive2 requests gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 6.7 MB/s eta 0:00:00


In [ ]:
# 🎨 RWC-AI-STUDIO - Complete AI Image Generation & Editing Studio
# Optimized for Google Colab with enhanced features and regeneration capabilities

import os
import re
import json
import time
import base64
import traceback
import gc
from io import BytesIO
from typing import List, Dict, Any, Tuple, Optional
from functools import wraps
from threading import Semaphore, Lock
import uuid
import datetime
import threading
import subprocess
import signal
import sys

# Google Colab specific imports
try:
    from google.colab import auth as colab_auth
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    colab_auth = None
    drive = None
    IN_COLAB = False

# Core dependencies
from flask import Flask, request, jsonify, render_template_string, send_file
from flask_cors import CORS
from werkzeug.utils import secure_filename
from PIL import Image, ImageDraw, ImageFont
import torch
import requests
from pyngrok import ngrok, conf

# AI/ML dependencies
from diffusers import (
    StableDiffusionImg2ImgPipeline,
    StableDiffusionPipeline,
    DDIMScheduler,
    EulerDiscreteScheduler,
    EulerAncestralDiscreteScheduler,
    DPMSolverMultistepScheduler
)
from transformers import logging as transformers_logging

try:
    from transformers import CLIPTextModel, CLIPTokenizer
    CLIP_AVAILABLE = True
except Exception:
    CLIPTextModel = None
    CLIPTokenizer = None
    CLIP_AVAILABLE = False

try:
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive
    PYDRIVE_AVAILABLE = True
except Exception:
    GoogleAuth = None
    GoogleDrive = None
    PYDRIVE_AVAILABLE = False

# Suppress transformer warnings
transformers_logging.set_verbosity_error()

print("🎨 RWC-AI-STUDIO - Initializing...")
print(f"📍 Running in {'Google Colab' if IN_COLAB else 'Local Environment'}")
print(f"🖥️ CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# =================== CONFIGURATION ===================

# Mount Google Drive if in Colab
if IN_COLAB:
    try:
        drive.mount('/content/drive')
        print("✅ Google Drive mounted successfully")
    except Exception as e:
        print(f"⚠️ Failed to mount Google Drive: {e}")

# Directory Configuration
BASE_DIR = "/content/drive/MyDrive/RWC-AI-STUDIO" if IN_COLAB else "./RWC-AI-STUDIO"
OUTPUT_DIR = os.path.join(BASE_DIR, "Generated")
INPUT_DIR = os.path.join(BASE_DIR, "Inputs")
META_DIR = os.path.join(BASE_DIR, "Metadata")
REGEN_DIR = os.path.join(BASE_DIR, "Regenerated")

# Create directories
for directory in [BASE_DIR, OUTPUT_DIR, INPUT_DIR, META_DIR, REGEN_DIR]:
    os.makedirs(directory, exist_ok=True)

# Model Configuration
MODEL_CONFIGS = {
    "dreamshaper": {
        "id": "Lykon/DreamShaper",
        "description": "Versatile model for artistic and realistic images",
        "type": "sd1.5"
    },
    "dreamlike": {
        "id": "dreamlike-art/dreamlike-diffusion-1.0",
        "description": "Ethereal and artistic image generation",
        "type": "sd1.5"
    },
    "runwayml": {
        "id": "runwayml/stable-diffusion-v1-5",
        "description": "Standard Stable Diffusion model",
        "type": "sd1.5"
    },
    "realistic": {
        "id": "SG161222/Realistic_Vision_V6.0_B1_noVAE",
        "description": "Photorealistic image generation",
        "type": "sd1.5"
    }
}

# Default Configuration
DEFAULT_MODEL = "dreamshaper"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
USE_FP16 = DEVICE == "cuda"
TORCH_DTYPE = torch.float16 if USE_FP16 else torch.float32

# API Configuration
API_KEY = os.environ.get("API_KEY", "rwc-ai-studio-2025")
HF_TOKEN = os.environ.get("HF_TOKEN", None)
NGROK_TOKEN = os.environ.get("NGROK_TOKEN", None)

# Upload limits
ALLOWED_EXTENSIONS = {'.png', '.jpg', '.jpeg', '.webp', '.bmp'}
MAX_UPLOAD_SIZE = 10 * 1024 * 1024  # 10MB
MAX_CONCURRENT_GENERATIONS = 1  # Prevent OOM

# Concurrency control
generation_semaphore = Semaphore(MAX_CONCURRENT_GENERATIONS)
model_lock = Lock()

# Style Presets
STYLE_PRESETS = {
    "photorealistic": {
        "positive": "photorealistic, highly detailed, professional photography, 8K, sharp focus",
        "negative": "cartoon, anime, painting, sketch, blurry, low quality"
    },
    "artistic": {
        "positive": "digital art, masterpiece, trending on artstation, highly detailed",
        "negative": "blurry, low quality, distorted"
    },
    "anime": {
        "positive": "anime style, manga, high quality, detailed, vibrant colors",
        "negative": "realistic, photographic, blurry, low quality"
    },
    "fantasy": {
        "positive": "epic fantasy, magical, ethereal, highly detailed, cinematic lighting",
        "negative": "modern, mundane, blurry, low quality"
    },
    "cyberpunk": {
        "positive": "cyberpunk, neon lights, futuristic, high tech, detailed",
        "negative": "natural, organic, blurry, low quality"
    }
}

# Generation Modes
GENERATION_MODES = {
    "text2img": {
        "name": "Text to Image",
        "description": "Generate images from text descriptions",
        "defaults": {"steps": 300, "guidance": 7.5, "width": 512, "height": 512}
    },
    "img2img": {
        "name": "Image to Image",
        "description": "Edit and transform existing images",
        "defaults": {"steps": 300, "guidance": 7.5, "strength": 0.7}
    },
    "enhance": {
        "name": "Enhance",
        "description": "Enhance image quality and details",
        "defaults": {"steps": 300, "guidance": 8.0, "strength": 0.4}
    },
    "regenerate": {
        "name": "Regenerate",
        "description": "Modify specific aspects of generated images",
        "defaults": {"steps": 183, "guidance": 7.0, "strength": 0.5}
    }
}

# =================== UTILITY FUNCTIONS ===================

def clear_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()

def generate_unique_id() -> str:
    """Generate unique 9-character ID"""
    timestamp = datetime.datetime.utcnow().strftime('%H%M')
    random_part = uuid.uuid4().hex[:5]
    return f"{timestamp}{random_part}"

def is_valid_file(filename: str) -> bool:
    """Check if file has allowed extension"""
    return '.' in filename and \
           os.path.splitext(filename)[1].lower() in ALLOWED_EXTENSIONS

def download_image_from_url(url: str) -> Optional[Image.Image]:
    """Download and process image from URL"""
    try:
        # Handle Google Drive URLs
        if "drive.google.com" in url:
            file_id = extract_drive_file_id(url)
            if file_id:
                url = f"https://drive.google.com/uc?export=download&id={file_id}"

        response = requests.get(url, timeout=30, headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
        response.raise_for_status()

        image = Image.open(BytesIO(response.content)).convert('RGB')
        return image
    except Exception as e:
        print(f"Error downloading image from {url}: {e}")
        return None

def extract_drive_file_id(url: str) -> Optional[str]:
    """Extract file ID from Google Drive URL"""
    patterns = [
        r'/file/d/([a-zA-Z0-9-_]+)',
        r'id=([a-zA-Z0-9-_]+)',
        r'/([a-zA-Z0-9-_]{25,})/'
    ]

    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    return None

def image_to_base64(image: Image.Image, format: str = 'PNG') -> str:
    """Convert PIL Image to base64 string"""
    buffer = BytesIO()
    image.save(buffer, format=format, quality=95 if format == 'JPEG' else None)
    buffer.seek(0)
    image_base64 = base64.b64encode(buffer.read()).decode('utf-8')
    return f"data:image/{format.lower()};base64,{image_base64}"

def save_image_with_metadata(image: Image.Image, metadata: Dict[str, Any]) -> str:
    """Save image with metadata"""
    unique_id = generate_unique_id()
    image_path = os.path.join(OUTPUT_DIR, f"{unique_id}.png")
    metadata_path = os.path.join(META_DIR, f"{unique_id}.json")

    # Save image & metadata
    image.save(image_path, 'PNG')

    metadata.update({
        'id': unique_id,
        'timestamp': datetime.datetime.utcnow().isoformat(),
        'image_path': image_path
    })

    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)

    return unique_id

# =================== MODEL MANAGEMENT ===================

class ModelManager:
    def __init__(self):
        self.loaded_models = {}
        self.current_model = None

    def load_model(self, model_name: str, model_type: str = "text2img") -> Optional[Any]:
        model_key = f"{model_name}_{model_type}"

        if model_key in self.loaded_models:
            return self.loaded_models[model_key]

        if model_name not in MODEL_CONFIGS:
            print(f"❌ Unknown model: {model_name}")
            return None

        config = MODEL_CONFIGS[model_name]

        try:
            print(f"📥 Loading {config['description']} ({model_type})...")
            clear_gpu_memory()

            # Load appropriate pipeline
            if model_type == "img2img":
                pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
                    config["id"],
                    torch_dtype=TORCH_DTYPE,
                    safety_checker=None,
                    requires_safety_checker=False,
                    use_auth_token=HF_TOKEN
                )
            else:
                pipeline = StableDiffusionPipeline.from_pretrained(
                    config["id"],
                    torch_dtype=TORCH_DTYPE,
                    safety_checker=None,
                    requires_safety_checker=False,
                    use_auth_token=HF_TOKEN
                )

            pipeline = pipeline.to(DEVICE)

            try:
                pipeline.enable_xformers_memory_efficient_attention()
            except Exception:
                try:
                    pipeline.enable_attention_slicing()
                except Exception:
                    pass

            try:
                pipeline.enable_sequential_cpu_offload()
            except Exception:
                pass

            self.loaded_models[model_key] = pipeline
            self.current_model = model_key

            print(f"✅ Model loaded successfully: {config['description']}")
            return pipeline

        except Exception as e:
            print(f"❌ Failed to load model {model_name}: {e}")
            clear_gpu_memory()
            return None

    def unload_model(self, model_key: str = None):
        if model_key is None:
            # Unload all models
            self.loaded_models.clear()
            self.current_model = None
        elif model_key in self.loaded_models:
            del self.loaded_models[model_key]
            if self.current_model == model_key:
                self.current_model = None

        clear_gpu_memory()
        print(f"🗑️ Model unloaded: {model_key or 'all models'}")

    def get_model_info(self) -> Dict[str, Any]:
        return {
            'loaded_models': list(self.loaded_models.keys()),
            'current_model': self.current_model,
            'available_models': MODEL_CONFIGS,
            'gpu_memory_allocated': torch.cuda.memory_allocated() if torch.cuda.is_available() else 0,
            'gpu_memory_reserved': torch.cuda.memory_reserved() if torch.cuda.is_available() else 0
        }

# Initialize model manager
model_manager = ModelManager()

# =================== GENERATION FUNCTIONS ===================

def generate_text_to_image(
    prompt: str,
    negative_prompt: str = "",
    model_name: str = DEFAULT_MODEL,
    style: str = "",
    width: int = 512,
    height: int = 512,
    steps: int = 300,
    guidance_scale: float = 7.5,
    seed: int = None
) -> Tuple[Optional[Image.Image], Dict[str, Any]]:
    """Generate image from text prompt"""

    metadata = {
        'mode': 'text2img',
        'prompt': prompt,
        'negative_prompt': negative_prompt,
        'model': model_name,
        'style': style,
        'width': width,
        'height': height,
        'steps': steps,
        'guidance_scale': guidance_scale,
        'seed': seed
    }

    try:
        with model_lock:
            pipeline = model_manager.load_model(model_name, "text2img")
            if pipeline is None:
                return None, {'error': f'Failed to load model: {model_name}'}

        # Apply style preset
        if style and style in STYLE_PRESETS:
            style_config = STYLE_PRESETS[style]
            prompt = f"{prompt}, {style_config['positive']}"
            if not negative_prompt:
                negative_prompt = style_config['negative']
            else:
                negative_prompt = f"{negative_prompt}, {style_config['negative']}"

        # Set seed for reproducibility
        if seed is not None:
            torch.manual_seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed(seed)

        # Generate image
        with generation_semaphore:
            result = pipeline(
                prompt=prompt,
                negative_prompt=negative_prompt if negative_prompt else None,
                width=width,
                height=height,
                num_inference_steps=steps,
                guidance_scale=guidance_scale,
                num_images_per_prompt=1
            )

        image = result.images[0]
        metadata['success'] = True

        return image, metadata

    except Exception as e:
        print(f"❌ Text2Image generation failed: {e}")
        clear_gpu_memory()
        return None, {'error': str(e)}

def generate_image_to_image(
    image: Image.Image,
    prompt: str,
    negative_prompt: str = "",
    model_name: str = DEFAULT_MODEL,
    style: str = "",
    strength: float = 0.7,
    steps: int = 300,
    guidance_scale: float = 7.5,
    seed: int = None
) -> Tuple[Optional[Image.Image], Dict[str, Any]]:
    """Transform existing image with text prompt"""

    metadata = {
        'mode': 'img2img',
        'prompt': prompt,
        'negative_prompt': negative_prompt,
        'model': model_name,
        'style': style,
        'strength': strength,
        'steps': steps,
        'guidance_scale': guidance_scale,
        'seed': seed,
        'original_size': image.size
    }

    try:
        with model_lock:
            pipeline = model_manager.load_model(model_name, "img2img")
            if pipeline is None:
                return None, {'error': f'Failed to load model: {model_name}'}

        if style and style in STYLE_PRESETS:
            style_config = STYLE_PRESETS[style]
            prompt = f"{prompt}, {style_config['positive']}"
            if not negative_prompt:
                negative_prompt = style_config['negative']
            else:
                negative_prompt = f"{negative_prompt}, {style_config['negative']}"

        # Resize image if too large
        max_size = 768
        if max(image.size) > max_size:
            ratio = max_size / max(image.size)
            new_size = tuple(int(dim * ratio) for dim in image.size)
            image = image.resize(new_size, Image.Resampling.LANCZOS)

        if seed is not None:
            torch.manual_seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed(seed)

        with generation_semaphore:
            result = pipeline(
                prompt=prompt,
                negative_prompt=negative_prompt if negative_prompt else None,
                image=image,
                strength=strength,
                num_inference_steps=steps,
                guidance_scale=guidance_scale,
                num_images_per_prompt=1
            )

        output_image = result.images[0]
        metadata['success'] = True
        metadata['output_size'] = output_image.size

        return output_image, metadata

    except Exception as e:
        print(f"❌ Img2Img generation failed: {e}")
        clear_gpu_memory()
        return None, {'error': str(e)}

def regenerate_image(
    original_image: Image.Image,
    modification_prompt: str,
    model_name: str = DEFAULT_MODEL,
    strength: float = 0.5,
    steps: int = 183,
    guidance_scale: float = 7.0,
    seed: int = None
) -> Tuple[Optional[Image.Image], Dict[str, Any]]:
    """Regenerate image with specific modifications"""

    metadata = {
        'mode': 'regenerate',
        'modification_prompt': modification_prompt,
        'model': model_name,
        'strength': strength,
        'steps': steps,
        'guidance_scale': guidance_scale,
        'seed': seed,
        'original_size': original_image.size
    }

    try:
        # Use img2img pipeline for regeneration
        result_image, gen_metadata = generate_image_to_image(
            image=original_image,
            prompt=modification_prompt,
            model_name=model_name,
            strength=strength,
            steps=steps,
            guidance_scale=guidance_scale,
            seed=seed
        )

        if result_image is not None:
            # Save to regenerated directory
            unique_id = generate_unique_id()
            regen_path = os.path.join(REGEN_DIR, f"{unique_id}.png")
            result_image.save(regen_path)

            metadata.update(gen_metadata)
            metadata['regenerated_path'] = regen_path
            metadata['success'] = True

        return result_image, metadata

    except Exception as e:
        print(f"❌ Image regeneration failed: {e}")
        return None, {'error': str(e)}

# =================== GOOGLE DRIVE INTEGRATION ===================

def setup_google_drive():
    if not PYDRIVE_AVAILABLE:
        print("⚠️ PyDrive2 not available - Google Drive features disabled")
        return None

    try:
        if IN_COLAB and colab_auth:
            colab_auth.authenticate_user()

        gauth = GoogleAuth()
        gdrive = GoogleDrive(gauth)
        print("✅ Google Drive integration ready")
        return gdrive
    except Exception as e:
        print(f"⚠️ Google Drive setup failed: {e}")
        return None

def upload_to_drive(file_path: str, gdrive_client, folder_name: str = "RWC-AI-STUDIO"):
    if gdrive_client is None:
        return {'error': 'Google Drive not initialized'}

    try:
        # Create or find folder
        folder_id = None
        file_list = gdrive_client.ListFile({'q': f"title='{folder_name}' and mimeType='application/vnd.google-apps.folder'"}).GetList()

        if file_list:
            folder_id = file_list[0]['id']
        else:
            folder = gdrive_client.CreateFile({'title': folder_name, 'mimeType': 'application/vnd.google-apps.folder'})
            folder.Upload()
            folder_id = folder['id']

        # Upload file
        file_metadata = {
            'title': os.path.basename(file_path),
            'parents': [{'id': folder_id}]
        }

        drive_file = gdrive_client.CreateFile(file_metadata)
        drive_file.SetContentFile(file_path)
        drive_file.Upload()

        # Make file public (optional)
        try:
            drive_file.InsertPermission({
                'type': 'anyone',
                'value': 'anyone',
                'role': 'reader'
            })
        except Exception:
            pass

        return {
            'success': True,
            'file_id': drive_file['id'],
            'view_link': f"https://drive.google.com/file/d/{drive_file['id']}/view",
            'download_link': f"https://drive.google.com/uc?id={drive_file['id']}"
        }

    except Exception as e:
        return {'error': str(e)}

# Initialize Google Drive
gdrive_client = setup_google_drive()

print("✅ RWC-AI-STUDIO core initialization complete")
print(f"📁 Base directory: {BASE_DIR}")
print(f"🤖 Available models: {', '.join(MODEL_CONFIGS.keys())}")
print(f"🎨 Available styles: {', '.join(STYLE_PRESETS.keys())}")

🎨 RWC-AI-STUDIO - Initializing...
📍 Running in Google Colab
🖥️ CUDA Available: True
🎯 GPU: Tesla T4
💾 GPU Memory: 14.7 GB
Mounted at /content/drive
✅ Google Drive mounted successfully
✅ Google Drive integration ready
✅ RWC-AI-STUDIO core initialization complete
📁 Base directory: /content/drive/MyDrive/RWC-AI-STUDIO
🤖 Available models: dreamshaper, dreamlike, runwayml, realistic
🎨 Available styles: photorealistic, artistic, anime, fantasy, cyberpunk


In [3]:
# =================== FLASK WEB APPLICATION ===================

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

# Security decorator
def require_api_key(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        api_key = request.headers.get('X-API-Key') or request.args.get('api_key')
        if not api_key or api_key != API_KEY:
            return jsonify({'error': 'Invalid API key'}), 401
        return f(*args, **kwargs)
    return decorated_function

def get_request_data(key: str, default=None):
    """Get data from JSON body, form data, or query parameters"""
    # Try JSON first
    if request.is_json:
        data = request.get_json(silent=True) or {}
        if key in data:
            return data[key]

    # Try form data
    if key in request.form:
        return request.form[key]

    # Try query parameters
    if key in request.args:
        return request.args[key]

    return default

# =================== API ROUTES ===================

@app.route('/')
def home():
    return render_template_string(WEB_INTERFACE)

@app.route('/health')
def health_check():
    return jsonify({
        'status': 'healthy',
        'version': '2.0.0',
        'timestamp': datetime.datetime.utcnow().isoformat(),
        'gpu_available': torch.cuda.is_available(),
        'models_loaded': len(model_manager.loaded_models)
    })

@app.route('/models', methods=['GET'])
def list_models():
    return jsonify({
        'available_models': MODEL_CONFIGS,
        'generation_modes': GENERATION_MODES,
        'style_presets': STYLE_PRESETS,
        'model_info': model_manager.get_model_info()
    })

@app.route('/generate/text2img', methods=['POST'])
@require_api_key
def api_generate_text2img():
    """Generate image from text"""
    try:
        # Get parameters
        prompt = get_request_data('prompt', '').strip()
        if not prompt:
            return jsonify({'error': 'Prompt is required'}), 400

        negative_prompt = get_request_data('negative_prompt', '').strip()
        model_name = get_request_data('model', DEFAULT_MODEL)
        style = get_request_data('style', '')
        width = int(get_request_data('width', 512))
        height = int(get_request_data('height', 512))
        steps = int(get_request_data('steps', 300))
        guidance_scale = float(get_request_data('guidance_scale', 7.5))
        seed = get_request_data('seed')
        if seed is not None:
            seed = int(seed)

        # Validate parameters
        width = max(64, min(1024, width))
        height = max(64, min(1024, height))
        steps = max(1, min(10000, steps))
        guidance_scale = max(1.0, min(20.0, guidance_scale))

        # Generate image
        result_image, metadata = generate_text_to_image(
            prompt=prompt,
            negative_prompt=negative_prompt,
            model_name=model_name,
            style=style,
            width=width,
            height=height,
            steps=steps,
            guidance_scale=guidance_scale,
            seed=seed
        )

        if result_image is None:
            return jsonify(metadata), 500

        # Save image with metadata
        image_id = save_image_with_metadata(result_image, metadata)

        # Upload to Google Drive if available
        drive_info = {}
        if gdrive_client:
            image_path = os.path.join(OUTPUT_DIR, f"{image_id}.png")
            drive_result = upload_to_drive(image_path, gdrive_client)
            if 'success' in drive_result:
                drive_info = drive_result

        return jsonify({
            'success': True,
            'image_id': image_id,
            'image_base64': image_to_base64(result_image),
            'metadata': metadata,
            'drive_info': drive_info
        })

    except Exception as e:
        print(f"❌ Text2Img API error: {e}")
        return jsonify({'error': str(e)}), 500

@app.route('/generate/img2img', methods=['POST'])
@require_api_key
def api_generate_img2img():
    """Transform existing image"""
    try:
        # Get input image
        input_image = None

        # Try file upload first
        if 'image' in request.files:
            file = request.files['image']
            if file and file.filename and is_valid_file(file.filename):
                input_image = Image.open(file.stream).convert('RGB')

        # Try image URL
        elif get_request_data('image_url'):
            image_url = get_request_data('image_url')
            input_image = download_image_from_url(image_url)

        if input_image is None:
            return jsonify({'error': 'Valid input image is required'}), 400

        # Get parameters
        prompt = get_request_data('prompt', '').strip()
        if not prompt:
            return jsonify({'error': 'Prompt is required'}), 400

        negative_prompt = get_request_data('negative_prompt', '').strip()
        model_name = get_request_data('model', DEFAULT_MODEL)
        style = get_request_data('style', '')
        strength = float(get_request_data('strength', 0.7))
        steps = int(get_request_data('steps', 300))
        guidance_scale = float(get_request_data('guidance_scale', 7.5))
        seed = get_request_data('seed')
        if seed is not None:
            seed = int(seed)

        # Validate parameters
        strength = max(0.1, min(1.0, strength))
        steps = max(1, min(10000, steps))
        guidance_scale = max(1.0, min(20.0, guidance_scale))

        # Generate image
        result_image, metadata = generate_image_to_image(
            image=input_image,
            prompt=prompt,
            negative_prompt=negative_prompt,
            model_name=model_name,
            style=style,
            strength=strength,
            steps=steps,
            guidance_scale=guidance_scale,
            seed=seed
        )

        if result_image is None:
            return jsonify(metadata), 500

        # Save image with metadata
        image_id = save_image_with_metadata(result_image, metadata)

        # Upload to Google Drive if available
        drive_info = {}
        if gdrive_client:
            image_path = os.path.join(OUTPUT_DIR, f"{image_id}.png")
            drive_result = upload_to_drive(image_path, gdrive_client)
            if 'success' in drive_result:
                drive_info = drive_result

        return jsonify({
            'success': True,
            'image_id': image_id,
            'image_base64': image_to_base64(result_image),
            'metadata': metadata,
            'drive_info': drive_info
        })

    except Exception as e:
        print(f"❌ Img2Img API error: {e}")
        return jsonify({'error': str(e)}), 500

@app.route('/regenerate', methods=['POST'])
@require_api_key
def api_regenerate_image():
    """Regenerate image with modifications"""
    try:
        # Get input image
        input_image = None

        # Try file upload first
        if 'image' in request.files:
            file = request.files['image']
            if file and file.filename and is_valid_file(file.filename):
                input_image = Image.open(file.stream).convert('RGB')

        # Try image URL
        elif get_request_data('image_url'):
            image_url = get_request_data('image_url')
            input_image = download_image_from_url(image_url)

        # Try existing image ID
        elif get_request_data('image_id'):
            image_id = get_request_data('image_id')
            image_path = os.path.join(OUTPUT_DIR, f"{image_id}.png")
            if os.path.exists(image_path):
                input_image = Image.open(image_path).convert('RGB')

        if input_image is None:
            return jsonify({'error': 'Valid input image is required'}), 400

        # Get modification parameters
        modification_prompt = get_request_data('modification_prompt', '').strip()
        if not modification_prompt:
            return jsonify({'error': 'Modification prompt is required'}), 400

        model_name = get_request_data('model', DEFAULT_MODEL)
        strength = float(get_request_data('strength', 0.5))
        steps = int(get_request_data('steps', 183))
        guidance_scale = float(get_request_data('guidance_scale', 7.0))
        seed = get_request_data('seed')
        if seed is not None:
            seed = int(seed)

        # Validate parameters
        strength = max(0.1, min(1.0, strength))
        steps = max(1, min(10000, steps))
        guidance_scale = max(1.0, min(20.0, guidance_scale))

        # Regenerate image
        result_image, metadata = regenerate_image(
            original_image=input_image,
            modification_prompt=modification_prompt,
            model_name=model_name,
            strength=strength,
            steps=steps,
            guidance_scale=guidance_scale,
            seed=seed
        )

        if result_image is None:
            return jsonify(metadata), 500

        # Save regenerated image
        image_id = save_image_with_metadata(result_image, metadata)

        # Upload to Google Drive if available
        drive_info = {}
        if gdrive_client:
            image_path = os.path.join(OUTPUT_DIR, f"{image_id}.png")
            drive_result = upload_to_drive(image_path, gdrive_client)
            if 'success' in drive_result:
                drive_info = drive_result

        return jsonify({
            'success': True,
            'image_id': image_id,
            'image_base64': image_to_base64(result_image),
            'metadata': metadata,
            'drive_info': drive_info
        })

    except Exception as e:
        print(f"❌ Regenerate API error: {e}")
        return jsonify({'error': str(e)}), 500

@app.route('/image/<image_id>')
def get_image(image_id):
    """Serve generated image"""
    try:
        image_path = os.path.join(OUTPUT_DIR, f"{image_id}.png")
        if os.path.exists(image_path):
            return send_file(image_path, mimetype='image/png')
        else:
            return jsonify({'error': 'Image not found'}), 404
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/metadata/<image_id>')
def get_metadata(image_id):
    """Get image metadata"""
    try:
        metadata_path = os.path.join(META_DIR, f"{image_id}.json")
        if os.path.exists(metadata_path):
            with open(metadata_path, 'r') as f:
                metadata = json.load(f)
            return jsonify(metadata)
        else:
            return jsonify({'error': 'Metadata not found'}), 404
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/gallery')
def gallery():
    """List generated images"""
    try:
        images = []
        for filename in os.listdir(OUTPUT_DIR):
            if filename.endswith('.png'):
                image_id = filename[:-4]
                metadata_path = os.path.join(META_DIR, f"{image_id}.json")

                image_info = {'id': image_id, 'filename': filename}

                if os.path.exists(metadata_path):
                    with open(metadata_path, 'r') as f:
                        metadata = json.load(f)
                    image_info['metadata'] = metadata

                images.append(image_info)

        # Sort by timestamp (newest first)
        images.sort(key=lambda x: x.get('metadata', {}).get('timestamp', ''), reverse=True)

        return jsonify({
            'images': images,
            'total': len(images)
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

print("✅ Flask API routes configured")


✅ Flask API routes configured


In [4]:
# =================== WEB INTERFACE ===================

WEB_INTERFACE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>🎨 RWC-AI-STUDIO</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            color: #333;
        }

        .container {
            max-width: 1400px;
            margin: 0 auto;
            padding: 20px;
        }

        .header {
            text-align: center;
            margin-bottom: 30px;
            color: white;
        }

        .header h1 {
            font-size: 3em;
            margin-bottom: 10px;
            text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
        }

        .header p {
            font-size: 1.2em;
            opacity: 0.9;
        }

        .main-content {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 30px;
            margin-bottom: 30px;
        }

        .panel {
            background: rgba(255, 255, 255, 0.95);
            border-radius: 15px;
            padding: 30px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.2);
            backdrop-filter: blur(10px);
        }

        .panel h2 {
            margin-bottom: 20px;
            color: #4a5568;
            border-bottom: 2px solid #667eea;
            padding-bottom: 10px;
        }

        .form-group {
            margin-bottom: 20px;
        }

        .form-group label {
            display: block;
            margin-bottom: 5px;
            font-weight: 600;
            color: #2d3748;
        }

        .form-control {
            width: 100%;
            padding: 12px;
            border: 2px solid #e2e8f0;
            border-radius: 8px;
            font-size: 14px;
            transition: border-color 0.3s;
        }

        .form-control:focus {
            outline: none;
            border-color: #667eea;
            box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.1);
        }

        textarea.form-control {
            resize: vertical;
            min-height: 100px;
        }

        .form-row {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 15px;
        }

        .btn {
            padding: 12px 30px;
            border: none;
            border-radius: 8px;
            font-size: 16px;
            font-weight: 600;
            cursor: pointer;
            transition: all 0.3s;
            text-transform: uppercase;
            letter-spacing: 1px;
        }

        .btn-primary {
            background: linear-gradient(45deg, #667eea, #764ba2);
            color: white;
        }

        .btn-primary:hover {
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(102, 126, 234, 0.4);
        }

        .btn-secondary {
            background: #718096;
            color: white;
        }

        .btn-secondary:hover {
            background: #4a5568;
        }

        .btn:disabled {
            opacity: 0.6;
            cursor: not-allowed;
            transform: none !important;
        }

        .mode-tabs {
            display: flex;
            margin-bottom: 20px;
            border-radius: 8px;
            overflow: hidden;
            background: #f7fafc;
        }

        .mode-tab {
            flex: 1;
            padding: 12px;
            text-align: center;
            cursor: pointer;
            transition: all 0.3s;
            font-weight: 600;
        }

        .mode-tab.active {
            background: #667eea;
            color: white;
        }

        .mode-tab:hover:not(.active) {
            background: #e2e8f0;
        }

        .result-panel {
            text-align: center;
        }

        .result-image {
            max-width: 100%;
            max-height: 500px;
            border-radius: 10px;
            box-shadow: 0 5px 20px rgba(0,0,0,0.2);
            margin-bottom: 20px;
        }

        .status {
            padding: 15px;
            border-radius: 8px;
            margin: 20px 0;
            font-weight: 600;
        }

        .status.loading {
            background: #bee3f8;
            color: #2c5282;
        }

        .status.success {
            background: #c6f6d5;
            color: #22543d;
        }

        .status.error {
            background: #fed7d7;
            color: #c53030;
        }

        .advanced-options {
            background: #f7fafc;
            border-radius: 8px;
            padding: 20px;
            margin-top: 20px;
        }

        .advanced-toggle {
            background: none;
            border: none;
            color: #667eea;
            cursor: pointer;
            font-weight: 600;
            margin-bottom: 15px;
        }

        .hidden {
            display: none;
        }

        .file-upload {
            border: 2px dashed #cbd5e0;
            border-radius: 8px;
            padding: 30px;
            text-align: center;
            transition: all 0.3s;
            cursor: pointer;
        }

        .file-upload:hover {
            border-color: #667eea;
            background: #f7fafc;
        }

        .file-upload.dragover {
            border-color: #667eea;
            background: #e6fffa;
        }

        .preview-image {
            max-width: 200px;
            max-height: 200px;
            border-radius: 8px;
            margin: 10px 0;
        }

        @media (max-width: 768px) {
            .main-content {
                grid-template-columns: 1fr;
            }

            .form-row {
                grid-template-columns: 1fr;
            }

            .mode-tabs {
                flex-direction: column;
            }
        }

        .spinner {
            border: 4px solid #f3f3f3;
            border-top: 4px solid #667eea;
            border-radius: 50%;
            width: 40px;
            height: 40px;
            animation: spin 2s linear infinite;
            margin: 20px auto;
        }

        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🎨 RWC-AI-STUDIO</h1>
            <p>Advanced AI Image Generation & Editing Studio</p>
        </div>

        <div class="main-content">
            <div class="panel">
                <h2>✨ Generate & Edit</h2>

                <div class="mode-tabs">
                    <div class="mode-tab active" onclick="switchMode('text2img')">Text → Image</div>
                    <div class="mode-tab" onclick="switchMode('img2img')">Image → Image</div>
                    <div class="mode-tab" onclick="switchMode('regenerate')">Regenerate</div>
                </div>

                <form id="generateForm">
                    <!-- Text2Img Mode -->
                    <div id="text2img-mode">
                        <div class="form-group">
                            <label for="prompt">Prompt *</label>
                            <textarea id="prompt" class="form-control" placeholder="Describe the image you want to generate..." required></textarea>
                        </div>

                        <div class="form-group">
                            <label for="negative_prompt">Negative Prompt</label>
                            <input type="text" id="negative_prompt" class="form-control" placeholder="What to avoid in the image...">
                        </div>

                        <div class="form-row">
                            <div class="form-group">
                                <label for="width">Width</label>
                                <select id="width" class="form-control">
                                    <option value="512" selected>512px</option>
                                    <option value="768">768px</option>
                                    <option value="1024">1024px</option>
                                </select>
                            </div>
                            <div class="form-group">
                                <label for="height">Height</label>
                                <select id="height" class="form-control">
                                    <option value="512" selected>512px</option>
                                    <option value="768">768px</option>
                                    <option value="1024">1024px</option>
                                </select>
                            </div>
                        </div>
                    </div>

                    <!-- Img2Img Mode -->
                    <div id="img2img-mode" class="hidden">
                        <div class="form-group">
                            <label>Input Image *</label>
                            <div class="file-upload" onclick="document.getElementById('imageInput').click()">
                                <p>📁 Click to upload or drag & drop an image</p>
                                <p style="color: #718096; font-size: 14px;">Supports PNG, JPG, JPEG, WebP</p>
                                <input type="file" id="imageInput" accept="image/*" style="display: none;">
                                <img id="imagePreview" class="preview-image hidden">
                            </div>
                        </div>

                        <div class="form-group">
                            <label for="img2img_prompt">Transformation Prompt *</label>
                            <textarea id="img2img_prompt" class="form-control" placeholder="How do you want to transform this image?"></textarea>
                        </div>

                        <div class="form-group">
                            <label for="img2img_negative">Negative Prompt</label>
                            <input type="text" id="img2img_negative" class="form-control" placeholder="What to avoid...">
                        </div>

                        <div class="form-group">
                            <label for="strength">Transformation Strength: <span id="strengthValue">0.7</span></label>
                            <input type="range" id="strength" class="form-control" min="0.1" max="1.0" step="0.1" value="0.7" oninput="updateStrengthValue()">
                            <small style="color: #718096;">Lower values preserve more of the original image</small>
                        </div>
                    </div>

                    <!-- Regenerate Mode -->
                    <div id="regenerate-mode" class="hidden">
                        <div class="form-group">
                            <label>Original Image *</label>
                            <div class="file-upload" onclick="document.getElementById('regenImageInput').click()">
                                <p>📁 Upload the image you want to modify</p>
                                <input type="file" id="regenImageInput" accept="image/*" style="display: none;">
                                <img id="regenImagePreview" class="preview-image hidden">
                            </div>
                        </div>

                        <div class="form-group">
                            <label for="modification_prompt">Modification Instructions *</label>
                            <textarea id="modification_prompt" class="form-control" placeholder="What specific changes do you want to make?"></textarea>
                        </div>

                        <div class="form-group">
                            <label for="regen_strength">Modification Strength: <span id="regenStrengthValue">0.5</span></label>
                            <input type="range" id="regen_strength" class="form-control" min="0.1" max="1.0" step="0.1" value="0.5" oninput="updateRegenStrengthValue()">
                        </div>
                    </div>

                    <!-- Common Options -->
                    <div class="form-row">
                        <div class="form-group">
                            <label for="model">AI Model</label>
                            <select id="model" class="form-control">
                                <option value="dreamshaper" selected>DreamShaper (Versatile)</option>
                                <option value="dreamlike">Dreamlike (Artistic)</option>
                                <option value="runwayml">RunwayML (Standard)</option>
                                <option value="realistic">Realistic Vision</option>
                            </select>
                        </div>
                        <div class="form-group">
                            <label for="style">Style Preset</label>
                            <select id="style" class="form-control">
                                <option value="">None</option>
                                <option value="photorealistic">Photorealistic</option>
                                <option value="artistic">Digital Art</option>
                                <option value="anime">Anime</option>
                                <option value="fantasy">Fantasy</option>
                                <option value="cyberpunk">Cyberpunk</option>
                            </select>
                        </div>
                    </div>

                    <div class="advanced-options">
                        <button type="button" class="advanced-toggle" onclick="toggleAdvanced()">⚙️ Advanced Settings</button>
                        <div id="advancedSettings" class="hidden">
                            <div class="form-row">
                                <div class="form-group">
                                    <label for="steps">Steps</label>
                                    <input type="number" id="steps" class="form-control" value="80" min="1" max="10000">
                                </div>
                                <div class="form-group">
                                    <label for="guidance_scale">Guidance Scale</label>
                                    <input type="number" id="guidance_scale" class="form-control" value="7.5" min="1" max="20" step="0.5">
                                </div>
                            </div>
                            <div class="form-group">
                                <label for="seed">Seed (optional)</label>
                                <input type="number" id="seed" class="form-control" placeholder="Random if empty">
                            </div>
                        </div>
                    </div>

                    <button type="submit" class="btn btn-primary" style="width: 100%; margin-top: 20px;">
                        🚀 Generate Image
                    </button>
                </form>
            </div>

            <div class="panel result-panel">
                <h2>🖼️ Result</h2>

                <div id="status" class="status hidden"></div>

                <div id="resultContainer" class="hidden">
                    <img id="resultImage" class="result-image">
                    <div style="margin-top: 20px;">
                        <button type="button" class="btn btn-secondary" onclick="downloadImage()">💾 Download</button>
                        <button type="button" class="btn btn-secondary" onclick="shareImage()">🔗 Share</button>
                    </div>
                </div>

                <div id="placeholderText" style="color: #a0aec0; font-style: italic; margin: 50px 0;">
                    Your generated image will appear here
                </div>
            </div>
        </div>
    </div>

    <script>
        const API_KEY = 'rwc-ai-studio-2025';
        let currentMode = 'text2img';
        let currentImageData = null;

        // Mode switching
        function switchMode(mode) {
            currentMode = mode;

            // Update tabs
            document.querySelectorAll('.mode-tab').forEach(tab => tab.classList.remove('active'));
            event.target.classList.add('active');

            // Show/hide mode sections
            document.getElementById('text2img-mode').classList.toggle('hidden', mode !== 'text2img');
            document.getElementById('img2img-mode').classList.toggle('hidden', mode !== 'img2img');
            document.getElementById('regenerate-mode').classList.toggle('hidden', mode !== 'regenerate');
        }

        // Advanced settings toggle
        function toggleAdvanced() {
            const settings = document.getElementById('advancedSettings');
            settings.classList.toggle('hidden');
        }

        // Strength value updates
        function updateStrengthValue() {
            document.getElementById('strengthValue').textContent = document.getElementById('strength').value;
        }

        function updateRegenStrengthValue() {
            document.getElementById('regenStrengthValue').textContent = document.getElementById('regen_strength').value;
        }

        // File upload handling
        function setupFileUpload(inputId, previewId) {
            const input = document.getElementById(inputId);
            const preview = document.getElementById(previewId);

            input.addEventListener('change', function(e) {
                const file = e.target.files[0];
                if (file) {
                    const reader = new FileReader();
                    reader.onload = function(e) {
                        preview.src = e.target.result;
                        preview.classList.remove('hidden');
                    };
                    reader.readAsDataURL(file);
                }
            });
        }

        setupFileUpload('imageInput', 'imagePreview');
        setupFileUpload('regenImageInput', 'regenImagePreview');

        // Status display
        function showStatus(message, type = 'loading') {
            const status = document.getElementById('status');
            status.textContent = message;
            status.className = `status ${type}`;
            status.classList.remove('hidden');
        }

        function hideStatus() {
            document.getElementById('status').classList.add('hidden');
        }

        // Form submission
        document.getElementById('generateForm').addEventListener('submit', async function(e) {
            e.preventDefault();

            const submitBtn = document.querySelector('button[type="submit"]');
            submitBtn.disabled = true;
            submitBtn.innerHTML = '<div class="spinner"></div> Generating...';

            showStatus('🎨 Generating your image...', 'loading');

            try {
                let response;

                if (currentMode === 'text2img') {
                    response = await generateText2Img();
                } else if (currentMode === 'img2img') {
                    response = await generateImg2Img();
                } else if (currentMode === 'regenerate') {
                    response = await regenerateImage();
                }

                if (response && response.success) {
                    displayResult(response);
                    showStatus('✅ Image generated successfully!', 'success');
                } else {
                    throw new Error(response?.error || 'Generation failed');
                }

            } catch (error) {
                console.error('Generation error:', error);
                showStatus(`❌ Error: ${error.message}`, 'error');
            } finally {
                submitBtn.disabled = false;
                submitBtn.innerHTML = '🚀 Generate Image';
            }
        });

        // API calls
        async function generateText2Img() {
            const data = {
                prompt: document.getElementById('prompt').value,
                negative_prompt: document.getElementById('negative_prompt').value,
                model: document.getElementById('model').value,
                style: document.getElementById('style').value,
                width: parseInt(document.getElementById('width').value),
                height: parseInt(document.getElementById('height').value),
                steps: parseInt(document.getElementById('steps').value),
                guidance_scale: parseFloat(document.getElementById('guidance_scale').value),
                seed: document.getElementById('seed').value || null
            };

            const response = await fetch('/generate/text2img', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                    'X-API-Key': API_KEY
                },
                body: JSON.stringify(data)
            });

            return await response.json();
        }

        async function generateImg2Img() {
            const formData = new FormData();
            const imageFile = document.getElementById('imageInput').files[0];

            if (!imageFile) {
                throw new Error('Please select an image file');
            }

            formData.append('image', imageFile);
            formData.append('prompt', document.getElementById('img2img_prompt').value);
            formData.append('negative_prompt', document.getElementById('img2img_negative').value);
            formData.append('model', document.getElementById('model').value);
            formData.append('style', document.getElementById('style').value);
            formData.append('strength', document.getElementById('strength').value);
            formData.append('steps', document.getElementById('steps').value);
            formData.append('guidance_scale', document.getElementById('guidance_scale').value);
            if (document.getElementById('seed').value) {
                formData.append('seed', document.getElementById('seed').value);
            }

            const response = await fetch('/generate/img2img', {
                method: 'POST',
                headers: {
                    'X-API-Key': API_KEY
                },
                body: formData
            });

            return await response.json();
        }

        async function regenerateImage() {
            const formData = new FormData();
            const imageFile = document.getElementById('regenImageInput').files[0];

            if (!imageFile) {
                throw new Error('Please select an image file');
            }

            formData.append('image', imageFile);
            formData.append('modification_prompt', document.getElementById('modification_prompt').value);
            formData.append('model', document.getElementById('model').value);
            formData.append('strength', document.getElementById('regen_strength').value);
            formData.append('steps', document.getElementById('steps').value);
            formData.append('guidance_scale', document.getElementById('guidance_scale').value);
            if (document.getElementById('seed').value) {
                formData.append('seed', document.getElementById('seed').value);
            }

            const response = await fetch('/regenerate', {
                method: 'POST',
                headers: {
                    'X-API-Key': API_KEY
                },
                body: formData
            });

            return await response.json();
        }

        // Display result
        function displayResult(result) {
            currentImageData = result;

            const resultImage = document.getElementById('resultImage');
            const resultContainer = document.getElementById('resultContainer');
            const placeholderText = document.getElementById('placeholderText');

            resultImage.src = result.image_base64;
            resultContainer.classList.remove('hidden');
            placeholderText.classList.add('hidden');
        }

        // Download image
        function downloadImage() {
            if (!currentImageData) return;

            const link = document.createElement('a');
            link.href = currentImageData.image_base64;
            link.download = `rwc-ai-studio-${currentImageData.image_id}.png`;
            document.body.appendChild(link);
            link.click();
            document.body.removeChild(link);
        }

        // Share image
        async function shareImage() {
            if (!currentImageData) return;

            if (navigator.share) {
                try {
                    // Convert base64 to blob for sharing
                    const response = await fetch(currentImageData.image_base64);
                    const blob = await response.blob();
                    const file = new File([blob], `rwc-ai-studio-${currentImageData.image_id}.png`, { type: 'image/png' });

                    await navigator.share({
                        title: 'RWC-AI-STUDIO Generated Image',
                        text: 'Check out this AI-generated image!',
                        files: [file]
                    });
                } catch (error) {
                    console.error('Sharing failed:', error);
                    copyImageLink();
                }
            } else {
                copyImageLink();
            }
        }

        function copyImageLink() {
            if (!currentImageData) return;

            const url = `${window.location.origin}/image/${currentImageData.image_id}`;
            navigator.clipboard.writeText(url).then(() => {
                showStatus('📋 Image link copied to clipboard!', 'success');
                setTimeout(hideStatus, 3000);
            });
        }

        // Drag and drop support
        function setupDragAndDrop() {
            const fileUploads = document.querySelectorAll('.file-upload');

            fileUploads.forEach(upload => {
                upload.addEventListener('dragover', (e) => {
                    e.preventDefault();
                    upload.classList.add('dragover');
                });

                upload.addEventListener('dragleave', () => {
                    upload.classList.remove('dragover');
                });

                upload.addEventListener('drop', (e) => {
                    e.preventDefault();
                    upload.classList.remove('dragover');

                    const files = e.dataTransfer.files;
                    if (files.length > 0) {
                        const input = upload.querySelector('input[type="file"]');
                        input.files = files;
                        input.dispatchEvent(new Event('change', { bubbles: true }));
                    }
                });
            });
        }

        // Initialize
        setupDragAndDrop();

        // Check API status on load
        fetch('/health')
            .then(response => response.json())
            .then(data => {
                console.log('RWC-AI-STUDIO Status:', data);
            })
            .catch(error => {
                console.error('API connection failed:', error);
            });
    </script>
</body>
</html>
"""

print("✅ Web interface template loaded")

✅ Web interface template loaded


In [ ]:
# =================== NGROK SETUP & SERVER LAUNCH ===================

def setup_ngrok():
    try:
        # Set ngrok auth token if provided
        if NGROK_TOKEN:
            ngrok.set_auth_token(NGROK_TOKEN)
            print("✅ ngrok auth token configured")
        else:
            print("⚠️ No ngrok token provided - using free tier (limited)")

        # Create tunnel
        print("🌐 Creating ngrok tunnel...")
        public_url = ngrok.connect(5000, bind_tls=True)

        print(f"🔗 Public URL: {public_url}")
        print(f"🔗 Local URL: http://localhost:5000")

        return public_url

    except Exception as e:
        print(f"❌ ngrok setup failed: {e}")
        print("💡 You can still use the local URL in Colab")
        return None

def launch_server():
    print("\n" + "="*60)
    print("🚀 LAUNCHING RWC-AI-STUDIO")
    print("="*60)

    # Load default model to ensure everything works
    print("📥 Loading default model...")
    default_pipeline = model_manager.load_model(DEFAULT_MODEL, "text2img")
    if default_pipeline:
        print(f"✅ Default model ({DEFAULT_MODEL}) loaded successfully")
    else:
        print(f"⚠️ Failed to load default model - will load on first request")

    # Setup ngrok tunnel
    public_url = setup_ngrok()

    print("\n" + "="*60)
    print("🎨 RWC-AI-STUDIO READY FOR USE!")
    print("="*60)
    print(f"💻 Local Server: http://localhost:5000")
    if public_url:
        print(f"🌐 Public Access: {public_url}")
    print(f"🔑 API Key: {API_KEY}")
    print(f"📁 Files saved to: {BASE_DIR}")
    print("\n🎯 Features Available:")
    print("   • Text-to-Image Generation")
    print("   • Image-to-Image Editing")
    print("   • Image Regeneration with Modifications")
    print("   • Multiple AI Models")
    print("   • Style Presets")
    print("   • Google Drive Integration")
    print("\n💡 Access the web interface using the URLs above!")
    print("="*60)

    # Start Flask server
    try:
        app.run(
            host='0.0.0.0',
            port=5000,
            debug=False,
            threaded=True,
            use_reloader=False
        )
    except KeyboardInterrupt:
        print("\n🛑 Server stopped by user")
    except Exception as e:
        print(f"❌ Server error: {e}")
    finally:
        # Cleanup
        try:
            ngrok.disconnect(public_url)
            print("✅ ngrok tunnel closed")
        except:
            pass

        print("🧹 Cleaning up GPU memory...")
        model_manager.unload_model()
        clear_gpu_memory()
        print("✅ Cleanup complete")

# Auto-launch when cell is executed
if __name__ == "__main__":
    launch_server()


## 🎯 How to Use RWC-AI-STUDIO

### 🚀 Quick Start
1. **Run all cells above** - This will install dependencies and launch the server
2. **Access the web interface** - Use the public URL provided by ngrok
3. **Choose your mode**:
   - **Text → Image**: Generate images from text descriptions
   - **Image → Image**: Transform existing images
   - **Regenerate**: Modify specific aspects of images

### 🎨 Features

**🤖 Multiple AI Models:**
- **DreamShaper**: Versatile model for various styles
- **Dreamlike**: Artistic and ethereal images
- **RunwayML**: Standard Stable Diffusion
- **Realistic Vision**: Photorealistic generation

**🎭 Style Presets:**
- Photorealistic
- Digital Art
- Anime
- Fantasy
- Cyberpunk

**✨ Key Features:**
- **Image Regeneration**: Modify specific aspects of generated images
- **Advanced Settings**: Control steps, guidance scale, seed
- **Google Drive Integration**: Automatic cloud backup
- **Public Access**: Share your studio via ngrok
- **Mobile Friendly**: Responsive web interface

### 🔧 Advanced Usage

**API Endpoints:**
- `POST /generate/text2img` - Text to image generation
- `POST /generate/img2img` - Image to image transformation
- `POST /regenerate` - Image regeneration with modifications
- `GET /gallery` - List all generated images
- `GET /models` - Available models and settings

**Tips for Best Results:**
1. **Be specific in prompts** - Detail helps generate better images
2. **Use negative prompts** - Specify what you don't want
3. **Experiment with strength** - Lower values preserve more original content
4. **Try different models** - Each has unique strengths
5. **Use seeds for consistency** - Same seed = reproducible results

### 🆘 Troubleshooting

**Common Issues:**
- **CUDA Out of Memory**: Reduce image size or use CPU
- **Model Loading Failed**: Check internet connection
- **Slow Generation**: Normal for high-quality images

**Performance Tips:**
- Use smaller image sizes for faster generation
- Lower step count for quicker results
- Close other GPU-intensive applications

---

**🎉 Enjoy creating amazing AI art with RWC-AI-STUDIO!**